<a href="https://colab.research.google.com/github/habibi-bd/computer-vision/blob/main/yolov9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade ultralytics


In [ ]:
# prompt: upzip

import zipfile
import os

# Assuming the zip file is named 'your_zip_file.zip' and is in the current directory
zip_file_path = '/content/alpaca_project.v2i.yolov9.zip'

# Define the directory where you want to extract the files
extract_dir = '/content/extracted_files'
os.makedirs(extract_dir, exist_ok=True) # Create the directory if it doesn't exist


# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# prompt: lets try to train the yolov9 using the dataset i uploaded

# Clone the YOLOv9 repository
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9

# Install the required dependencies
!pip install -r requirements.txt -q

# Download pre-trained weights (e.g., yolov9-e.pt)
!wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

# Define the path to your data.yaml file (assuming it's in the extracted directory)
data_yaml_path = '/content/extracted_files/data.yaml'

# Define the path to your desired output directory
output_dir = '/content/yolov9_training_output'
os.makedirs(output_dir, exist_ok=True)


!python train.py --batch 16 --epochs 100 --data {data_yaml_path} --cfg models/detect/yolov9-e.yaml --weights yolov9-e.pt --name yolov9_custom --cache




In [ ]:
from ultralytics import YOLO

model = YOLO("yolov9c.pt")

model.train(
    data="/content/extracted_files/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16
)


In [ ]:
# Your model is saved automatically here:
"/content/sample_data"

# Rename and move it anywhere you want:
import shutil
shutil.copy("runs/detect/train/weights/best.pt", "/content/sample_data")


In [ ]:
import shutil


import os


with open("/content/extracted_files/data.yaml") as f:
    print(f.read())


In [ ]:
metrics = model.val()

# Print the key performance metrics
print("📊 Accuracy Results:")
print("mAP@0.5     :", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)

      # mean recall, float


In [ ]:
from IPython.display import Image
Image(filename='runs/detect/train/results.png')


In [ ]:
results = model.predict(source='/content/Alpaca_(31562329701).jpg', conf=0.25)


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Get the annotated image (BGR format)
annotated_img = results[0].plot()

# Convert BGR to RGB for matplotlib
annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

# Show image
plt.figure(figsize=(10, 8))
plt.imshow(annotated_img)
plt.axis('off')
plt.title('YOLOv9 Prediction')
plt.show()


In [ ]:
results=model.train(
    data="/content/extracted_files/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    augment=True,     # Enable built-in augmentations
    mosaic=True,      # Enable mosaic augmentation (combining 4 images)
    mixup=True,       # Enable mixup augmentation (blend images)
    hsv_h=0.015,      # Hue augmentation range
    hsv_s=0.7,        # Saturation augmentation range
    hsv_v=0.4,        # Value augmentation range
    flipud=0.0,       # Vertical flip probability (0 means off)
    fliplr=0.5,       # Horizontal flip probability (default 0.5)
    degrees=0.0,      # Random rotation degrees
    translate=0.1,    # Random translation fraction
    scale=0.5,        # Random scale fraction
    shear=0.0         # Shear angle degrees
)


In [ ]:
predictions = model.predict('/content/Alpaca_(31562329701).jpg', conf=0.25)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics.utils.plotting import Annotator


annotated_img = predictions[0].plot()  # returns image with boxes drawn (BGR)

# Convert BGR to RGB for correct color display in matplotlib
annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

# Plot the image
plt.figure(figsize=(10, 8))
plt.imshow(annotated_img)
plt.axis('off')
plt.title('YOLOv9 Prediction with Confidence')
plt.show()